# Модель "мешок слов"


Пусть есть коллекция документов $D$. Будем рассматривать модель *bag-of-words* (мешок слов), то есть каждый документ состоит из какого-то набора слов (терма) без учета их позиций внутри документа. 

Рассмотрим коллекцию документов:

In [1]:
docs = ['человек лев орел черепаха человек', 
        'лев вол орел',
        'лев черепаха лев кошка',
        'жучка кошка мышка',
        'лев орел грифон']

Посчитаем сколько каждое слово встретилось во всей коллекции и сколько каждом документе:

In [2]:
import re
import numpy as np
import pandas as pd


from collections import defaultdict, Counter 

def parse_doc(doc):
    return re.split(r'\s+', doc, re.U)

# подсчет df по коллекции
def calc_df_dict(docs):
    c = Counter() 
    for doc_id, doc in enumerate(docs):
        c.update(set(parse_doc(doc)))
    return c 
    
# подсчет tf для документа
def calc_tf_dict(doc):
    c = Counter() 
    for word in parse_doc(doc):
        c[word] += 1
    return c 
    
dfs = calc_df_dict(docs)
pd.DataFrame(data=list(dfs.items()), columns=['term', 'df'])

,term,df
0,орел,3
1,лев,4
2,человек,1
3,черепаха,2
4,вол,1
5,кошка,2
6,жучка,1
7,мышка,1
8,грифон,1


Каждому слову и каждому документу можно присвоить уникальный численный идентификатор, и построить так называемую tf-матрицу, состоящую из элементов $\{tf_{t,d}\}$ - вес слова $t$ в документе $d$ (где $t = 1 \dots n$ - индексы слов, $j = 1 \dots k$ - индексы документов). Под весом может подразумеваться число вхождений, нормализированная частота, и т.п.

### Система обозначений SMART

$\{tf_{t,d}\}$ не всегда дает адекватное представление (документы могут быть сильно не равной длины, не учитывается значимость слов). Поэтому существует более сложные модели под обобщенным названием *tf-idf* (term frequency - inverted document frequency), где используется документная частота слова. Идея в том, что для каждого слова в каждом документе считается *tf*, потом *idf* и значения перемножаются. Потом для каждого документа получается вектор, с которым можно что-то сделать (нормализовать).

| Частота термина                                                                                                  | Документная частота                                            | Нормировка                                              |
|------------------------------------------------------------------------------------------------------------------|----------------------------------------------------------------|---------------------------------------------------------|
| n $$	\text{tf}_{t,d}$$                                                                                             | n $$1$$                                                        | n $$1$$                                                 |
| l $$1 + \log{\text{tf}_{t,d}}$$                                                                                  | t $$\log{\frac{N}{\text{df}_i}}$$                              | c $$\frac{1}{\sqrt{\omega_1^2 + \ldots + \omega_m^2}}$$ |
| a $$0.5 + \frac{0.5 \text{tf}_{t,d}}{\max_t{ \text{tf}_{t,d}}}$$                                                 | p $$\text{max}(0, \log{\frac{N - \text{df}_i}{\text{df}_i}})$$ |
| b $$ 1,  \text{if }  \text{tf}_{t,d} > 0 \text{ else } 0 $$                                                              |
| L $$\frac{1 + \log{\text{tf}_{t,d}}}{1 + \log{\text{avg}_{t \in d}(\text{tf}_{t,d})}}$$                          |


### Ипользование scikit-learn

In [3]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer

vectorizer = CountVectorizer()
vectorizer.fit_transform(docs).toarray()  

array([[0, 0, 0, 0, 1, 0, 1, 2, 1],
       [1, 0, 0, 0, 1, 0, 1, 0, 0],
       [0, 0, 0, 1, 2, 0, 0, 0, 1],
       [0, 0, 1, 1, 0, 1, 0, 0, 0],
       [0, 1, 0, 0, 1, 0, 1, 0, 0]])

In [4]:
print('\n'.join(vectorizer.get_feature_names_out()))

вол
грифон
жучка
кошка
лев
мышка
орел
человек
черепаха


## Использование в кластеризации, классификации и тематическом моделировании

Напишем код для чтения и обработки коллекции новостей

In [5]:
import gzip

from dataclasses import dataclass
from typing import Iterator

from nltk.corpus import stopwords
from yargy.tokenizer import MorphTokenizer


@dataclass
class Text:
    label: str
    title: str
    text: str


def read_texts(fn: str) -> Iterator[Text]:
    with gzip.open(fn, "rt", encoding="utf-8") as f:
        for line in f:
            yield Text(*line.strip().split("\t"))


tokenizer = MorphTokenizer()
ru_stopwords = set(stopwords.words("russian"))


def normalize_text(text: str) -> str:
    tokens = [
        tok.normalized for tok in tokenizer(text) if tok.normalized not in ru_stopwords
    ]
    return " ".join(tokens)

Прочитаем текст и преобразуем документы в вектора

In [6]:
texts = list(read_texts("data/news.txt.gz"))

vectorizer = TfidfVectorizer(max_df=0.2, min_df=10)
# vectorizer = HashingVectorizer()

X = vectorizer.fit_transform([normalize_text(text.text) for text in texts]).toarray()

понизим размерность c помощью `PCA`

In [7]:
from sklearn.decomposition import PCA

pca = PCA(n_components=500)
X = pca.fit_transform(X)

напишем кол для преобразования новых текстов

In [8]:
def transform_text(text: str) -> np.ndarray:
    normalized_text = normalize_text(text)
    vect = vectorizer.transform([normalized_text]).toarray()
    return pca.transform(vect)

transform_text("привет миру")

array([[-3.58265835e-02, -6.62540416e-03, -1.04680806e-02,
        -5.41690602e-04, -1.14059274e-02, -2.40370263e-02,
         5.48597548e-03,  1.62393436e-02, -7.78476070e-03,
        -3.61940841e-03,  8.34901066e-03,  4.51348383e-02,
         4.80463555e-02,  1.74930757e-02, -6.02130162e-03,
         2.47462130e-02,  1.34572823e-03, -2.11793647e-04,
         6.17791054e-04, -9.55725071e-03,  1.61799990e-02,
         6.19196328e-03,  2.15180773e-02, -1.36050896e-02,
        -6.04139021e-03, -3.87056555e-02,  1.49080920e-02,
         5.57994597e-03, -7.54837189e-05,  1.28803812e-03,
         6.92215555e-03, -1.62097659e-02, -7.98437529e-04,
         1.43853901e-03, -5.05733510e-03, -1.43767224e-03,
        -3.52559811e-02,  2.94052598e-02, -4.41110305e-04,
         5.23548383e-03,  8.80641411e-04, -1.83856098e-02,
        -1.72274201e-02,  4.51181651e-02,  3.57792774e-03,
        -6.67035272e-03, -8.28719714e-03,  3.96747931e-03,
         3.18318969e-02,  2.07871811e-02, -4.89994196e-0

Кластеризация

In [9]:
from sklearn.cluster import KMeans

k_means = KMeans(n_clusters=10, n_init="auto")
k_means.fit(X)

KMeans(n_clusters=10, n_init='auto')

Классификация

In [10]:
from sklearn.svm import SVC

y = [text.label for text in texts]

svc = SVC()
svc.fit(X, y)

SVC()

In [20]:
svc.predict(transform_text("Футбольное Динамо забило гол и выиграло"))

array(['sport'], dtype='<U9')

Тематическое моделирование

In [22]:
import re

from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaModel

normalized_tokens = [
    re.findall(r"\b\w+\b", normalize_text(text.text))
    for text in texts
]
dictionary = Dictionary(normalized_tokens)

corpus = [dictionary.doc2bow(text) for text in normalized_tokens]

lda = LdaModel(corpus, num_topics=10)

In [32]:
id2token = {v : k for (k, v) in dictionary.token2id.items()}
[(id2token[token_id], p) for (token_id, p) in lda.get_topic_terms(9, 20)]

[('который', 0.012838349),
 ('год', 0.010673094),
 ('учёный', 0.00916909),
 ('это', 0.006336624),
 ('свой', 0.0062552043),
 ('человек', 0.005108341),
 ('мочь', 0.0043260455),
 ('также', 0.0041958876),
 ('исследование', 0.004040174),
 ('новый', 0.0037101565),
 ('время', 0.0035189835),
 ('журнал', 0.0034221103),
 ('стать', 0.0031218128),
 ('модель', 0.0029651897),
 ('однако', 0.002866491),
 ('сообщать', 0.002820199),
 ('опубликовать', 0.0026814279),
 ('работа', 0.0026671356),
 ('являться', 0.0025679762),
 ('весь', 0.0025233657)]